In [1]:
# ==================== IMPORTS ========================
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage
from pydantic import BaseModel, Field
from langchain_community.tools import DuckDuckGoSearchRun

# =================== MODELS ============================
# Generator Model (tool-enabled)
generator_model = ChatOllama(
    model="llama3.2:1b",
    temperature=0.7
)

# Evaluator Output Schema
class BlogEvaluate(BaseModel):
    evaluation: Literal["approved", "needs_improvement"] = Field(
        ...,
        description="Final assessment indicating whether the blog meets required standards"
    )
    feedback: str = Field(
        ...,
        description="Concise, actionable feedback highlighting strengths and improvements"
    )
    score: int = Field(
        ...,
        ge=0,
        le=10,
        description="Overall quality score from 0 to 10"
    )

# Evaluator Model
evaluator_model = ChatOllama(
    model="phi3:mini",
    temperature=0.5
)
structured_evaluator_model = evaluator_model.with_structured_output(BlogEvaluate)

# Optimizer Model
optimizer_model = ChatOllama(
    model="qwen2.5:0.5b",
    temperature=0.3
)

# ===================== TOOLS =========================
search_tool = DuckDuckGoSearchRun(name="Search")

# ==================== STATE ==========================
class BlogState(TypedDict):
    topic: str
    search: str
    blog: str
    evaluation: Literal["approved", "needs_improvement"]
    feedback: str
    score: int
    iteration: int
    max_iteration: int

# ===================== NODES ==========================
def search_topic(state: BlogState):
    """Fetch latest information from the internet"""
    query = f"Latest updates, statistics, reforms, and developments about {state['topic']}"
    results = search_tool.run(query)
    return {"search": results}


def generate_blog(state: BlogState):
    prompt = [
        SystemMessage(
            content="You are a professional blog writer producing accurate, structured, factual blogs."
        ),
        HumanMessage(content=f"""
Topic: {state['topic']}

Latest Reference Information:
{state['search']}

Write a professional blog following this structure:
- Introduction
- Background / Overview
- Core Discussion with sub-sections
- Challenges or Limitations
- Best Practices or Key Considerations
- Conclusion with clear takeaways

Rules:
- Use the latest information provided
- No Q&A format
- Under 4000 characters
- Clear headings and professional tone
- Version {state['iteration']}
""")
    ]
    response = generator_model.invoke(prompt).content
    return {"blog": response}


def evaluate_blog(state: BlogState):
    prompt = [
        SystemMessage(
            content="You are a strict professional evaluator validating quality and factual accuracy."
        ),
        HumanMessage(content=f"""
Blog Content:
{state['blog']}

Latest Reference Information:
{state['search']}

Evaluate based on:
1. Topic relevance
2. Structure and clarity
3. Depth and insight
4. Practical value
5. Accuracy vs latest information
6. Professional tone
7. Length compliance

Reject if:
- Facts contradict reference info
- Weak structure or filler content
- Generic or empty conclusion

Respond ONLY in structured format.
""")
    ]
    result = structured_evaluator_model.invoke(prompt)
    return {
        "evaluation": result.evaluation,
        "feedback": result.feedback,
        "score": result.score
    }


def optimize_blog(state: BlogState):
    prompt = [
        SystemMessage(
            content="You improve blogs strictly using evaluator feedback and factual references."
        ),
        HumanMessage(content=f"""
Topic: {state['topic']}

Latest Reference Information:
{state['search']}

Evaluator Feedback:
{state['feedback']}

Original Blog:
{state['blog']}

Improve clarity, depth, accuracy, and structure.
Keep under 4000 characters.
Avoid repetition and filler.
""")
    ]
    improved_blog = optimizer_model.invoke(prompt).content
    return {
        "blog": improved_blog,
        "iteration": state["iteration"] + 1
    }


def route_evaluation(state: BlogState):
    if state["evaluation"] == "approved":
        return "approved"
    if state["iteration"] >= state["max_iteration"]:
        return "approved"
    return "needs_improvement"

# ======================= GRAPH =============================
graph = StateGraph(BlogState)

graph.add_node("search", search_topic)
graph.add_node("generate", generate_blog)
graph.add_node("evaluate", evaluate_blog)
graph.add_node("optimize", optimize_blog)

graph.add_edge(START, "search")
graph.add_edge("search", "generate")
graph.add_edge("generate", "evaluate")

graph.add_conditional_edges(
    "evaluate",
    route_evaluation,
    {
        "approved": END,
        "needs_improvement": "optimize"
    }
)

graph.add_edge("optimize", "evaluate")

workflow = graph.compile()

# ======================== RUN =========================
initial_state: BlogState = {
    "topic": "Indian Railways",
    "search": "",
    "blog": "",
    "evaluation": "needs_improvement",
    "feedback": "",
    "score": 0,
    "iteration": 1,
    "max_iteration": 5
}

final_state = workflow.invoke(initial_state)

print("\n===== FINAL BLOG =====\n")
print(final_state["blog"])

print("\n===== SEARCH REFERENCES USED =====\n")
print(final_state["search"])

print("\n===== EVALUATION =====")
print(f"Status: {final_state['evaluation']}")
print(f"Score: {final_state['score']}/10")
print(f"Feedback: {final_state['feedback']}")



===== FINAL BLOG =====

**Introduction**

Indian Railways is one of the largest and most iconic transportation systems in the world. With a rich history spanning over a century, it has revolutionized the way people travel across the country. From its humble beginnings to its current status as a modern marvel, Indian Railways continues to evolve and improve its services.

**Background/Overview**

Established in 1853, Indian Railways is a subsidiary of the Indian Government. Over the years, it has expanded exponentially to become one of the largest rail networks globally. With over 1.5 million kilometers of tracks, Indian Railways connects major cities, towns, and villages across India.

**Core Discussion**

### Signaling & Telecom

Indian Railways plans to upgrade its signaling system to automatic signaling in 15,000 km. This will enhance track maintenance efficiency and reduce the frequency of shutdowns.

### Digitalization

The railways are investing heavily in digitalization initiat